# CEFR Basics Tutorial

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/engineerinvestor/financial-health-calculator/blob/main/examples/01_cefr_basics.ipynb)

This notebook introduces the **Certainty-Equivalent Funded Ratio (CEFR)**, a metric for measuring how well-funded your retirement is after accounting for:
- Taxes
- Liquidity constraints
- Concentration risk

In [ ]:
# Install the fundedness package
!pip install fundedness -q

In [ ]:
# Import required modules
from fundedness import (
    Asset, BalanceSheet, Liability, compute_cefr, CEFRResult
)
from fundedness.models.assets import AccountType, AssetClass, LiquidityClass, ConcentrationLevel
from fundedness.models.liabilities import LiabilityType, InflationLinkage
from fundedness.models.tax import TaxModel
from fundedness.viz.waterfall import create_cefr_waterfall

## 1. Define Your Assets

Let's create a sample portfolio with different account types:

In [ ]:
# Create assets
assets = [
    Asset(
        name="401(k)",
        value=500_000,
        account_type=AccountType.TAX_DEFERRED,
        asset_class=AssetClass.STOCKS,
        liquidity_class=LiquidityClass.RETIREMENT,
        concentration_level=ConcentrationLevel.DIVERSIFIED,
    ),
    Asset(
        name="Roth IRA",
        value=200_000,
        account_type=AccountType.TAX_EXEMPT,
        asset_class=AssetClass.STOCKS,
        liquidity_class=LiquidityClass.RETIREMENT,
        concentration_level=ConcentrationLevel.DIVERSIFIED,
    ),
    Asset(
        name="Taxable Brokerage",
        value=300_000,
        account_type=AccountType.TAXABLE,
        asset_class=AssetClass.STOCKS,
        liquidity_class=LiquidityClass.TAXABLE_INDEX,
        concentration_level=ConcentrationLevel.DIVERSIFIED,
        cost_basis=200_000,  # $100k in unrealized gains
    ),
    Asset(
        name="Cash",
        value=50_000,
        account_type=AccountType.TAXABLE,
        asset_class=AssetClass.CASH,
        liquidity_class=LiquidityClass.CASH,
        concentration_level=ConcentrationLevel.DIVERSIFIED,
    ),
]

balance_sheet = BalanceSheet(assets=assets)
print(f"Total Assets: ${balance_sheet.total_value:,.0f}")

## 2. Define Your Spending

Now let's define your planned spending (liabilities):

In [ ]:
# Create liabilities
liabilities = [
    Liability(
        name="Essential Living Expenses",
        liability_type=LiabilityType.ESSENTIAL_SPENDING,
        annual_amount=50_000,
        is_essential=True,
        inflation_linkage=InflationLinkage.CPI,
    ),
    Liability(
        name="Travel & Hobbies",
        liability_type=LiabilityType.DISCRETIONARY_SPENDING,
        annual_amount=20_000,
        is_essential=False,
        inflation_linkage=InflationLinkage.CPI,
    ),
]

total_spending = sum(l.annual_amount for l in liabilities)
print(f"Total Annual Spending: ${total_spending:,.0f}")

## 3. Calculate CEFR

The CEFR formula is:

$$\text{CEFR} = \frac{\sum(\text{Asset} \times (1-\tau) \times \lambda \times \rho)}{PV(\text{Liabilities})}$$

Where:
- $\tau$ = tax rate
- $\lambda$ = liquidity factor
- $\rho$ = reliability factor

In [ ]:
# Calculate CEFR
result = compute_cefr(
    balance_sheet=balance_sheet,
    liabilities=liabilities,
    planning_horizon=30,  # 30-year retirement
    real_discount_rate=0.02,
    base_inflation=0.025,
)

print(f"CEFR: {result.cefr:.2f}")
print(f"Funded: {result.is_funded}")
print(f"\n{result.get_interpretation()}")

## 4. Understand the Haircuts

Let's see how much each type of haircut reduces your effective assets:

In [ ]:
print(f"Gross Assets:           ${result.gross_assets:>12,.0f}")
print(f"Tax Haircut:            ${result.total_tax_haircut:>12,.0f} ({result.total_tax_haircut/result.gross_assets*100:.1f}%)")
print(f"Liquidity Haircut:      ${result.total_liquidity_haircut:>12,.0f} ({result.total_liquidity_haircut/result.gross_assets*100:.1f}%)")
print(f"Reliability Haircut:    ${result.total_reliability_haircut:>12,.0f} ({result.total_reliability_haircut/result.gross_assets*100:.1f}%)")
print(f"                        {'-'*12}")
print(f"Net Assets:             ${result.net_assets:>12,.0f}")
print(f"\nLiability PV:           ${result.liability_pv:>12,.0f}")
print(f"Funding Gap:            ${result.funding_gap:>12,.0f}")

## 5. Visualize with Waterfall Chart

In [ ]:
# Create waterfall chart
fig = create_cefr_waterfall(result)
fig.show()

## 6. Asset-by-Asset Breakdown

In [ ]:
import pandas as pd

# Create detailed breakdown
data = []
for d in result.asset_details:
    data.append({
        "Asset": d.asset.name,
        "Gross Value": f"${d.gross_value:,.0f}",
        "Tax Rate": f"{d.tax_rate:.1%}",
        "Liquidity": f"{d.liquidity_factor:.0%}",
        "Reliability": f"{d.reliability_factor:.0%}",
        "Net Value": f"${d.net_value:,.0f}",
        "Total Haircut": f"{d.total_haircut:.1%}",
    })

pd.DataFrame(data)

## 7. What-If Analysis

Let's see how changing assets or spending affects CEFR:

In [ ]:
# What if we have 20% more assets?
higher_assets = [Asset(**{**a.model_dump(), "value": a.value * 1.2}) for a in assets]
higher_result = compute_cefr(
    balance_sheet=BalanceSheet(assets=higher_assets),
    liabilities=liabilities,
    planning_horizon=30,
)
print(f"With 20% more assets: CEFR = {higher_result.cefr:.2f}")

# What if we reduce spending by 20%?
lower_spending = [Liability(**{**l.model_dump(), "annual_amount": l.annual_amount * 0.8}) for l in liabilities]
lower_result = compute_cefr(
    balance_sheet=balance_sheet,
    liabilities=lower_spending,
    planning_horizon=30,
)
print(f"With 20% less spending: CEFR = {lower_result.cefr:.2f}")

## Summary

Key takeaways:

1. **CEFR >= 1.0** means you're fully funded
2. **Tax haircuts** vary by account type (Roth = 0%, Traditional = high)
3. **Liquidity haircuts** penalize illiquid assets like real estate
4. **Reliability haircuts** penalize concentrated positions

Use CEFR to understand your true financial position, not just your gross portfolio value!